Let's begin to investigate how an imperfect drift-scan observation strategy will
affect the reconstruction of foreground alm.

In [1]:
from pygdsm import GlobalSkyModel2016
import healpy as hp
import numpy as np
from scipy.special import eval_legendre
import matplotlib.pyplot as plt
import seaborn as sns

import src.beam_functions as BF
import src.spherical_harmonics as SH
import src.forward_model as FM
import src.sky_models as SM
import src.map_making as MM

RS = SH.RealSphericalHarmonics()